In [1]:
import numpy as np

import gym

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticCnnPolicy, ActorCriticPolicy
from stable_baselines3.common.utils import get_schedule_fn

/root/.cache/pypoetry/virtualenvs/root-qv7zuvpL-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
class MyEnv(gym.Env):
    def __init__(self):
        self.state_dim = 5
        self.action_dim = 3
        self.action_space = gym.spaces.Discrete(self.action_dim) 
        
        #状態が3つの時で上限と下限の設定と仮定
        LOW = np.zeros(self.state_dim).astype(float)
        HIGH = np.ones(self.state_dim).astype(float)
        self.observation_space = gym.spaces.Box(low=LOW, high=HIGH)
        self.n_steps = 0

    def reset(self):
        self.n_steps = 0
        observation = np.random.rand(self.state_dim)
        return observation

    def step(self, action_index: int):
        self.n_steps += 1
        if action_index == 1:
            reward = 1
        else:
            reward = -1
        observation = np.random.rand(self.state_dim)
        done = self.n_steps > 10
        return observation, reward, done, {"episode": {"r": 1, "l": 1}}

    def render(self):
        raise NotImplementedError()

In [17]:
state_dim = 5
action_dim = 3


state_space = gym.spaces.Box(low=np.zeros(state_dim).astype(float), high=np.ones(state_dim).astype(float))
action_space = gym.spaces.Discrete(action_dim)

env = MyEnv()

In [18]:
from pathlib import Path
from stable_baselines3.common.logger import configure

curdir = Path().resolve()
logdir = curdir / "log"
logdir.mkdir(parents=True, exist_ok=True)

new_logger = configure(str(logdir), ["stdout", "csv"])

model = PPO(policy=ActorCriticPolicy, env=env, verbose=1)
model.set_logger(new_logger)

Logging to /home/opt/notobooks/ppo/log
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [19]:

model.learn(total_timesteps=25000, eval_env=env, eval_freq=2048)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Eval num_timesteps=2048, episode_reward=-7.00 +/- 2.53
Episode length: 11.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 11       |
|    mean_reward     | -7       |
| time/              |          |
|    total_timesteps | 2048     |
---------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.9      |
|    ep_rew_mean     | 0.62     |
| time/              |          |
|    fps             | 4964     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
Eval num_timesteps=4096, episode_reward=11.00 +/- 0.00
Episode length: 11.00 +/- 0.00
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 11         |
|    mean_reward          | 

In [55]:
obs = env.reset()
i = 0
actions = list()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, _ = env.step(action)
    actions.append(action)
    # env.render()
    i += 1
    if i > 100:
        break